In [164]:
#Libs
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [165]:
import spacy
#!python -m spacy download en_core_web_lg

nlp = spacy.load("en_core_web_lg")

In [444]:
#data
dim_cd_df = pd.read_csv('../data/interim_data/dim_cd_df.csv',index_col=0)
dim_qq_df = pd.read_csv('../data/interim_data/dim_qq_df.csv',index_col=0)
FACT_df = pd.read_csv('../data/interim_data/FACT_df.csv',index_col=0)


In [623]:
#Subset the Fact table for only data from the US for 2020 and remove rows where the answer was NA
us_data = pd.merge(FACT_df,dim_cd_df[['Account_Num','Country','geometry']],on='Account_Num')
us_data.drop_duplicates(inplace=True)
us_data=us_data.query("Country == 'United States of America' & Year ==2020 & Answer != 'Question not applicable'")

In [624]:
#merge with dim_qq_df for the question section
us_data = pd.merge(us_data,dim_qq_df[['Question_ID','Sect']],on='Question_ID')

In [625]:
#sample 5000 rows
us_data_sample = us_data.sample(5000).reset_index(drop=True)
us_data_sample.head()

,Account_Num,Year,Question_ID,Answer,Country,geometry,Sect
0,58627,2020,5811,Increased demand for public services,United States of America,POINT (-90.1843 38.8906),Climate Hazards
1,58357,2020,20952,10,United States of America,POINT (-118.362 34.09),Energy
2,49339,2020,16845,Economic growth,United States of America,POINT (157.59 21.28),Mitigation Actions
3,32550,2020,20697,2030,United States of America,POINT (-104.985 39.7376),Energy
4,50560,2020,6376,High,United States of America,POINT (-122.271 37.8044),Climate Hazards


In [652]:
import plotly.express as px

In [653]:
def vectorize(answer):
    doc=nlp(answer)
    return doc.vector

In [654]:
doc = list(nlp.pipe(us_data_sample.Answer))

In [655]:
vectors=[tokens.vector for tokens in doc]

In [656]:
import warnings
warnings.filterwarnings("ignore")

In [657]:
#isometric 

#from article

### Step 1 - Configure the Isomap function, note we use default hyperparameter values in this example
embed3 = Isomap(
    n_neighbors=5, # default=5, algorithm finds local structures based on the nearest neighbors
    n_components=3, # number of dimensions
    eigen_solver='auto', # {‘auto’, ‘arpack’, ‘dense’}, default=’auto’
    tol=0, # default=0, Convergence tolerance passed to arpack or lobpcg. not used if eigen_solver == ‘dense’.
    max_iter=None, # default=None, Maximum number of iterations for the arpack solver. not used if eigen_solver == ‘dense’.
    path_method='auto', # {‘auto’, ‘FW’, ‘D’}, default=’auto’, Method to use in finding shortest path.
    neighbors_algorithm='auto', # neighbors_algorithm{‘auto’, ‘brute’, ‘kd_tree’, ‘ball_tree’}, default=’auto’
    n_jobs=-1, # n_jobsint or None, default=None, The number of parallel jobs to run. -1 means using all processors
    metric='minkowski', # string, or callable, default=”minkowski”
    p=2, # default=2, Parameter for the Minkowski metric. When p = 1, this is equivalent to using manhattan_distance (l1), and euclidean_distance (l2) for p = 2
    metric_params=None # default=None, Additional keyword arguments for the metric function.
)

### Step 2 - Fit the data and transform it, so we have 3 dimensions instead of 64
X_trans3 = embed3.fit_transform(vectors)
    
### Step 3 - Print shape to test
print('The new shape of X: ',X_trans3.shape)

The new shape of X:  (5000, 3)


In [658]:
y=testing_data.Sect
y.shape

(5000,)

In [661]:
fig.update_layout?

In [660]:


# Create a 3D scatter plot
fig = px.scatter_3d(None, 
                    x=X_trans3[:,0], 
                    y=X_trans3[:,1], 
                    z=X_trans3[:,2],
                    color=y.astype(str),
                    height=900, width=900
                   )

# Update chart looks
fig.update_layout(#title_text="Scatter 3D Plot",
                  showlegend=True,
                  legend=dict(orientation="h", y=0, xanchor="center", x=0.5),
                  scene_camera=dict(up=dict(x=0, y=0, z=1), 
                                        center=dict(x=0, y=0, z=-0.2),
                                        eye=dict(x=-1.5, y=1.5, z=0.5)),
                                        margin=dict(l=0, r=0, b=0, t=0),
                  scene = dict(xaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='#f0f0f0',
                                          title_font=dict(size=10),
                                          tickfont=dict(size=10),
                                         ),
                               yaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='#f0f0f0',
                                          title_font=dict(size=10),
                                          tickfont=dict(size=10),
                                          ),
                               zaxis=dict(backgroundcolor='lightgrey',
                                          color='black', 
                                          gridcolor='#f0f0f0',
                                          title_font=dict(size=10),
                                          tickfont=dict(size=10),
                                         )))

# Update marker size
fig.update_traces(marker=dict(size=2))

fig.show()

In [622]:
#Add States to the model. The problem is we don't have state info in our main DBs so Austin, for example, could be associated with the numerous Austin's in different States